In [1]:
import numpy_financial as npf

class ProjectFinance:
    """
    A class to calculate NPV, IRR, PI, Payback, and Discounted Payback for a project.
    """
    def __init__(self, name, cash_flows):
        self.name = name
        self.cash_flows = cash_flows
        self.initial_investment = abs(cash_flows[0]) # Stored as positive cost
        self.future_cash_flows = cash_flows[1:]
        
    def calculate_npv(self, rate):
        """Calculates Net Present Value (NPV)."""
        return npf.npv(rate, self.cash_flows)

    def calculate_irr(self):
        """Calculates Internal Rate of Return (IRR)."""
        return npf.irr(self.cash_flows)

    def calculate_pi(self, rate):
        """Calculates Profitability Index (PI)."""
        # Calculate the PV of future cash inflows (CF1 through CF4)
        # We use [0] prefix to align cash flows for the npf.npv function which treats the first element as CF0
        pv_inflows = npf.npv(rate, [0] + self.future_cash_flows)
        
        if self.initial_investment == 0:
            return float('inf')
        return pv_inflows / self.initial_investment

    # --- NEW METHOD: Calculate Annual Discounted Cash Flows (DCF) ---
    def calculate_dcf(self, rate):
        """
        Calculates the Present Value (Discounted Cash Flow) for each year's inflow.
        Returns a list of discounted future cash flows (excluding Year 0).
        """
        dcf_list = []
        for t, cf in enumerate(self.future_cash_flows, start=1):
            dcf = cf / ((1 + rate) ** t)
            dcf_list.append(dcf)
        return dcf_list
    
    # --- NEW METHOD: Payback Period (PB) ---
    def calculate_payback_period(self):
        """Calculates the standard Payback Period."""
        remaining_cost = self.initial_investment
        cumulative_cf = 0
        
        for t, cf in enumerate(self.future_cash_flows, start=1):
            cumulative_cf += cf
            if cumulative_cf >= self.initial_investment:
                # Payback = Year before recovery + (Remaining Cost / CF in Recovery Year)
                # Since remaining_cost is the cost *before* this year's cash flow, we use it directly.
                fraction = (remaining_cost / cf)
                return t - 1 + fraction
            remaining_cost -= cf
        return float('inf') # Never pays back

    # --- NEW METHOD: Discounted Payback Period (DPB) ---
    def calculate_discounted_payback_period(self, rate):
        """Calculates the Discounted Payback Period."""
        # Get the PV of future cash flows
        dcf_list = self.calculate_dcf(rate)
        
        remaining_cost = self.initial_investment
        cumulative_dcf = 0
        
        for t, dcf in enumerate(dcf_list, start=1):
            cumulative_dcf += dcf
            if cumulative_dcf >= self.initial_investment:
                # DPB = Year before recovery + (Remaining Discounted Cost / DCF in Recovery Year)
                fraction = (remaining_cost / dcf)
                return t - 1 + fraction
            remaining_cost -= dcf
        return float('inf') # Never pays back

# --- Project Data ---
project_s_flows = [-1000, 500, 400, 300, 100]
project_l_flows = [-1000, 100, 300, 400, 600]

# --- Calculations ---
DISCOUNT_RATE = 0.10 # 10%

project_s = ProjectFinance("Project S", project_s_flows)
project_l = ProjectFinance("Project L", project_l_flows)

# --- Results ---
results = []
for project in [project_s, project_l]:
    npv = project.calculate_npv(DISCOUNT_RATE)
    irr = project.calculate_irr()
    pi = project.calculate_pi(DISCOUNT_RATE)
    pb = project.calculate_payback_period()
    dpb = project.calculate_discounted_payback_period(DISCOUNT_RATE)
    dcf_list = project.calculate_dcf(DISCOUNT_RATE)
    
    results.append({
        "Project": project.name,
        "NPV": f"${npv:,.2f}",
        "IRR": f"{irr * 100:.2f}%",
        "PI": f"{pi:.3f}",
        "Payback Period": f"{pb:.2f} years",
        "Discounted Payback Period": f"{dpb:.2f} years",
        "DCF (PV of Inflows)": dcf_list
    })

# --- Output ---
print(f"--- Financial Metrics Summary (Discount Rate: {DISCOUNT_RATE*100:.0f}%) ---")
for result in results:
    print(f"\n{result['Project']}:")
    print(f"  NPV: {result['NPV']}")
    print(f"  IRR: {result['IRR']}")
    print(f"  PI:  {result['PI']}")
    print(f"  Payback Period: {result['Payback Period']}")
    print(f"  Discounted Payback: {result['Discounted Payback Period']}")
    print(f"  Yearly DCF (CF1-CF4): {['$' + f'{dcf:.2f}' for dcf in result['DCF (PV of Inflows)']]}" )

--- Financial Metrics Summary (Discount Rate: 10%) ---

Project S:
  NPV: $78.82
  IRR: 14.49%
  PI:  1.079
  Payback Period: 2.33 years
  Discounted Payback: 2.95 years
  Yearly DCF (CF1-CF4): ['$454.55', '$330.58', '$225.39', '$68.30']

Project L:
  NPV: $49.18
  IRR: 11.79%
  PI:  1.049
  Payback Period: 3.33 years
  Discounted Payback: 3.88 years
  Yearly DCF (CF1-CF4): ['$90.91', '$247.93', '$300.53', '$409.81']
